In [4]:
import numpy as np
import os
from components.network import Helper, Network

positives = Helper.get_positives(number_of_samples=500, mel_spectrogram=False)
filepath = os.path.join(os.path.pardir, 'data/train_test_file_small.npz')
np.savez(filepath, positives=positives)

C:\Users\grodo\Miniconda3\envs\Musicc\lib\site-packages\librosa\core\audio.py:162: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
C:\Users\grodo\Miniconda3\envs\Musicc\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Length of the specimen should be 660984


In [5]:
import os
import numpy as np
from components.network import Helper

filepath = os.path.join(os.path.pardir, 'data/train_test_file_small.npz')
file = np.load(filepath, allow_pickle=True)
positives = file['positives']
negatives = Helper.get_negatives(500, sample_length=660719, mel_spectrogram=False)
train, test = Helper.prepare_dataset(positives, negatives, train_to_test_ratio=0.8)
train_mel = Helper.convert_to_mel_spectrograms(train)
test_mel = Helper.convert_to_mel_spectrograms(test)
x_train_mel, y_train_mel = Helper.get_x_y(train_mel)
x_test_mel, y_test_mel = Helper.get_x_y(test_mel)

train_tempo = Helper.convert_to_tempogram(train)
test_tempo = Helper.convert_to_tempogram(test)
x_train_tempo, y_train_tempo = Helper.get_x_y(train_tempo)
x_test_tempo, y_test_tempo = Helper.get_x_y(test_tempo)


Number of all negative samples: 6965
Length of the specimen should be 660719


In [6]:
from components.network import Network

network_mel = Network(input_shape=x_train_mel[0].shape)
network_tempo = Network(input_shape=x_train_tempo[0].shape)

In [1]:
import os
from components.network import Network

network_mel = Network(input_shape=(128, 1291))
network_tempo = Network(input_shape=(384, 1291))
network_mel.load_model(os.path.join(os.path.pardir, 'models/model_mel'))
network_tempo.load_model(os.path.join(os.path.pardir, 'models/model_tempo'))

Using plaidml.keras.backend backend.
INFO:plaidml:Opening device "opencl_amd_gfx1010.0"


In [7]:
network_mel.train(x_train_mel, y_train_mel, epochs=25, batch_size=100)
network_tempo.train(x_train_tempo, y_train_tempo, epochs=25, batch_size=100)

network_mel.evaluate(x_test_mel, y_test_mel)
network_tempo.evaluate(x_test_tempo, y_test_tempo)

Epoch 1/25
800/800 [==============================] - 40s 51ms/step - loss: 5.5171 - acc: 0.8975
Epoch 2/25
800/800 [==============================] - 1s 793us/step - loss: 0.1000 - acc: 0.9925

(0.0826918876171112, 0.99)

In [8]:
import os

network_mel.save_model(os.path.join(os.path.pardir, f'models/model_mel'))
network_tempo.save_model(os.path.join(os.path.pardir, f'models/model_tempo'))

In [ ]:
import librosa

from components.specimen import Evolution

#specimen_length = 660984
specimen_length = 660719
evolution = Evolution(neural_networks=[network_mel, network_tempo],
                      features_list=[librosa.feature.melspectrogram, librosa.feature.tempogram], number_of_specimen=100,
                      specimen_length=specimen_length, target=None, mutation_chance=0.1, crossover_chance=0.1)
evolution.run_epochs(number_of_epochs=100, save=True, save_as_negative=False, epsilon=0.2)

Started fitness calculation0


In [3]:
evolution.run_epochs(number_of_epochs=100, save=True, save_as_negative=False, epsilon=0.2)



Started epoch 1
Started fitness calculation
Finished fitness calculation
Best fitness: 1.0
Worst fitness: 0.0
Saved file: neg_1332.wav
Saved file: neg_1333.wav
Saved file: neg_1334.wav
Saved file: neg_1335.wav
Saved file: neg_1336.wav
Saved file: neg_1337.wav
Saved file: neg_1338.wav
Saved file: neg_1339.wav
Saved file: neg_1340.wav
Saved file: neg_1341.wav
Started epoch 2
Started fitness calculation
Finished fitness calculation
Best fitness: 1.0
Worst fitness: 0.0
Saved file: neg_1342.wav
Started epoch 3
Started fitness calculation
Finished fitness calculation
Best fitness: 1.0
Worst fitness: 0.0
Saved file: neg_1343.wav
Started epoch 4
Started fitness calculation
Finished fitness calculation
Best fitness: 1.0
Worst fitness: 0.0
Saved file: neg_1344.wav
Saved file: neg_1345.wav
Saved file: neg_1346.wav
Started epoch 5
Started fitness calculation
Finished fitness calculation
Best fitness: 1.0
Worst fitness: 0.0
Saved file: neg_1347.wav
Saved file: neg_1348.wav
Saved file: neg_1349.wav
